# Explore the COS dark parameter space
Author: Jo Taylor (jotaylor@stsci.edu)

Last Update: Mar 19 2024

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display("text/html", "<style>.container { width:100% !important; }</style>")

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread, spread
#from holoviews import opts
#from holoviews.selection import link_selections
hv.extension('bokeh')
import panel as pn

from acdc.database.query_cos_dark import counts_by_mjd, all_darks
from acdc.database import within_saa

print(hv.__version__)

### Get exposure information for all darks, by default
To connect to your local database, add the `dbname=` option when invoking `all_darks()`. This should include the full path of the database, e.g. `/path/to/cos_dark.db`. If in the current directory, do not include `.` or `./`, simply use `cos_dark.db`.

You may also specify selection criteria by using `counts_by_mjd()`.

In [ ]:
#df = counts_by_mjd(mjdstart=56500, mjdend=56510)
df = all_darks()
df.sort_values(by=["expstart"], inplace=True)

Flag data that was taken within the SAA.

In [ ]:
flags = within_saa.saaFilter(df["longitude"].to_numpy(), df["latitude"].to_numpy())
df_flag = df.loc[flags]
df_noflag = df.loc[flags == False]

### Create the interactive exploration plot
First define the holoviews widget- you should not edit anything here.

In [ ]:
import param
import panel as pn

class DarkExplorer(param.Parameterized):
    flagsaa = param.Boolean(default=False)
    region = param.ObjectSelector(default="inner", objects=set(df["region"]))
    segment = param.ObjectSelector(default="FUVA", objects=["FUVA", "FUVB"])
    hv = param.ObjectSelector(default=167, objects=set(df["hv"]))
    
    xaxis = param.ObjectSelector(default="expstart", objects=df.columns.to_list())
    yaxis = param.ObjectSelector(default="dark_pha11", objects=df.columns.to_list())
    
    @param.depends("region", "segment", "hv", "xaxis", "yaxis")
    def saa_false(self):
        dfplot_noflag = df_noflag.loc[(df_noflag["segment"] == self.segment) & (df_noflag["hv"] == self.hv) & (df_noflag["region"] == self.region)]
        sctr_noflag = hv.Scatter((dfplot_noflag[self.xaxis], dfplot_noflag[self.yaxis]))
        return sctr_noflag
    
    @param.depends("region", "segment", "hv", "xaxis", "yaxis")
    def saa_true(self):
        dfplot_flag = df_flag.loc[(df_flag["segment"] == self.segment) & (df_flag["hv"] == self.hv) & (df_flag["region"] == self.region)]
        sctr_flag = hv.Scatter((dfplot_flag[self.xaxis], dfplot_flag[self.yaxis]))
        return sctr_flag
    
    @param.depends("flagsaa")
    def view(self):
        dmap_noflag = hv.DynamicMap(self.saa_false)
        #dmap_ds = dynspread(datashade(dmap_noflag)).opts(framewise=True, height=700, width=1000)
        dmap_ds = dmap_noflag.opts(framewise=True, height=700, width=1000)
        if self.flagsaa:
            dmap_flag = hv.DynamicMap(self.saa_true)
            #dmap_flag_ds = dynspread(datashade(dmap_flag, cmap=["red"])).opts(framewise=True, height=700, width=1000)
            dmap_flag_ds = dmap_flag.opts(color="red", framewise=True, height=700, width=1000)
            dmap_ds = dmap_ds * dmap_flag_ds
        return dmap_ds
            

### Display the widget. 
Click the `Flagsaa` box to display data taken with the SAA as red points. If unchecked, data within SAA *will not be displayed*.

The `Region`, `Segment`, and `Hv` selections determine which data will be plotted.

The `Xaxis`, `Yaxis` selections determine what arrays are plotted on the x and y axes.

**NOTE:** There may be a small delay (1-2 seconds) when choosing a new data subselection to plot.

In [ ]:
explorer = DarkExplorer()
app = pn.Row(explorer.param, explorer.view)
app

### Not in use

This displays the widget using `dynspread`, which responds to data reselection much faster, but the plot quality is greatly degraded.

In [ ]:
#explorer = DarkExplorer()
#dark_dmap = dynspread(datashade(hv.DynamicMap(explorer.load_data).opts(framewise=True), cmap=["cornflowerblue"]).opts(framewise=True), threshold=0.5).opts(framewise=True, height=700, width=1000)
#dark_dmap = hv.DynamicMap(explorer.load_data)
#pn.Row(pn.panel(explorer.param, parameters=["flagsaa", "region", "segment", "hv", "xaxis", "yaxis"]), dark_dmap)